In [1]:
#Setting up the code and installing required packages 
from util import *
from glob import glob
import matplotlib.pyplot as plt
from shapely import wkt
from shapely.geometry import LineString, Polygon
pd.set_option("display.max_columns", None)

In [2]:
#Navigate to intersects shapefile and separates out the year, finds how many years from 1800 and how many years to 2100
gdf = gpd.read_file(f"Data/Merged Intersects_UniqueID/JackettIsland_Intersects.shp")
gdf["Date"] = pd.to_datetime(gdf.ShorelineI, dayfirst=True, format='mixed')
gdf["Year"] = gdf.Date.dt.year
gdf["YearsSinceBase"] = (gdf.Date - pd.Timestamp(1800, 1, 1)).dt.days / 365.25
gdf["YearsUntilFuture"] = (
    pd.Timestamp(2100, 1, 1) - gdf.Date
    ).dt.days / 365.25
gdf.Date = gdf.Date.astype(str)
gdf["TransectID"] = gdf.Unique_ID.astype(int)
gdf

,ShorelineI,BaselineID,Distance,IntersectX,IntersectY,Uncertaint,Unique_ID,Date,geometry,Year,YearsSinceBase,YearsUntilFuture,TransectID
0,04/19/2017,1.0,-97.80,1.602805e+06,5.444480e+06,2.94,2.010001e+11,2017-04-19,POINT Z (1602805.403 5444479.642 0.000),2017,217.292266,82.702259,-2147483648
1,01/13/2000,1.0,-54.62,1.602842e+06,5.444502e+06,5.32,2.010001e+11,2000-01-13,POINT Z (1602842.047 5444502.478 0.000),2000,200.027379,99.967146,-2147483648
2,02/19/2010,1.0,-69.27,1.602830e+06,5.444495e+06,2.32,2.010001e+11,2010-02-19,POINT Z (1602829.612 5444494.729 0.000),2010,210.130048,89.864476,-2147483648
3,09/13/1985,1.0,-54.69,1.602842e+06,5.444502e+06,4.33,2.010001e+11,1985-09-13,POINT Z (1602841.988 5444502.442 0.000),1985,185.694730,114.299795,-2147483648
4,02/04/1967,1.0,-65.04,1.602833e+06,5.444497e+06,5.94,2.010001e+11,1967-04-02,POINT Z (1602833.209 5444496.970 0.000),1967,167.244353,132.750171,-2147483648
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2337,02/22/1940,3.0,-29.51,1.603246e+06,5.444123e+06,3.77,2.010002e+11,1940-02-22,POINT Z (1603246.456 5444123.385 0.000),1940,140.136893,159.857632,-2147483648
2338,06/25/2013,3.0,-113.28,1.603204e+06,5.444051e+06,2.30,2.010002e+11,2013-06-25,POINT Z (1603203.881 5444051.243 0.000),2013,213.475702,86.518823,-2147483648
2339,08/28/2006,3.0,-109.38,1.603206e+06,5.444055e+06,2.32,2.010002e+11,2006-08-28,POINT Z (1603205.860 5444054.597 0.000),2006,206.650240,93.344285,-2147483648
2340,01/31/1980,3.0,-115.24,1.603203e+06,5.444050e+06,5.31,2.010002e+11,1980-01-31,POINT Z (1603202.885 5444049.556 0.000),1980,180.076660,119.917864,-2147483648


In [3]:
def get_transects(intersects):
  p1 = intersects.geometry[intersects.Distance.idxmin()].coords[0]
  p2 = intersects.geometry[intersects.Distance.idxmax()].coords[0]
  azimuth = math.degrees(math.atan2(p1[0]-p2[0], p1[1]-p2[1]))
  if azimuth < 0:
      azimuth += 360
  return pd.Series({"Azimuth": azimuth, "geometry": LineString([p1, p2])})

lines = gdf.groupby("TransectID").apply(get_transects, include_groups=False)
lines.crs = gdf.crs
lines

,Azimuth,geometry
TransectID,,
-2147483648,333.024904,"LINESTRING Z (1603278.890 5443963.291 0.000, 1..."


In [4]:
lines["dist_to_neighbour"] = lines.distance(lines.shift(-1))
breakpoints = lines.dist_to_neighbour[lines.dist_to_neighbour > 105]
lines["group"] = pd.Series(range(len(breakpoints)), index=breakpoints.index)
lines["group"] = lines.group.bfill().fillna(len(breakpoints)).astype(int)
transect_metadata = lines[["Azimuth", "group"]].to_dict(orient="index")

C:\Users\lalit\AppData\Local\Temp\ipykernel_23428\1477094512.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  lines["group"] = lines.group.bfill().fillna(len(breakpoints)).astype(int)


In [5]:
#Linear regression is run here. See util.py for the breakdown on linear_models
linear_models = fit(gdf, transect_metadata)
linear_models

,TransectID,slope,intercept,group,r2_score,mae,mse,rmse
0,-2147483648,-0.119532,-53.507306,0,0.029211,14.119566,331.024402,18.194076


In [ ]:
#Only run if rolling average is needed. Otherwise SKIP THIS
#linear_models = fit(gdf, transect_metadata)
#rolled_slopes = linear_models.groupby("group").slope.rolling(10, min_periods=1).mean().dropna().reset_index(level=0)
#linear_models.slope = rolled_slopes.slope
#linear_models.dropna(inplace=True)
#linear_models

In [6]:
#Coordinates of the projected shoreline are plotted here

#Changed coordinate function by making old_x and old_y negative 
def calculate_new_coordinates(old_x, old_y, bearing, distance):
    bearing_radians = math.radians(bearing)
    new_x = old_x + (distance * math.sin(bearing_radians))
    new_y = old_y + (distance * math.cos(bearing_radians))
    point = Point(new_x, new_y)
    assert not point.is_empty
    return point

#Removed other model equations and changed Azimuth addtion from 180 to 360 deg
def predict(
    df: pd.DataFrame,
    linear_models: pd.DataFrame,
    transect_metadata: dict,
):
    """_summary_

    Args:
        df (pd.DataFrame): dataframe with columns: TransectID, Date, Distance, YearsSinceBase
        linear_models (pd.DataFrame): dataframe with columns: TransectID, slope, intercept
        transect_metadata (dict): dict lookup of TransectID to Azimuth & group
        
    Returns:
        pd.DataFrame: resulting prediction points for the year 2100
    """
    results = []
    for i, row in linear_models.iterrows():
        transect_ID = row.TransectID
        transect_df = df[df.TransectID == transect_ID]
        latest_row = transect_df[transect_df.Date == transect_df["Date"].max()].iloc[0]
        future_year = int(row.get("FUTURE_YEAR", FUTURE_YEAR))
        result = row.to_dict()
        result.update({
            "TransectID": transect_ID,
            "BaselineID": latest_row.BaselineID,
            "group": row.group,
            "Year": future_year,
            "ocean_point": calculate_new_coordinates(
                latest_row.geometry.x,
                latest_row.geometry.y,
                transect_metadata[transect_ID]["Azimuth"] + 180,
                500,
            ),
        })
        
        model = "linear"
        slope = row.slope
        intercept = row.intercept

        predicted_distance = slope * (future_year - 1800) + intercept
        distance_difference = latest_row.Distance - predicted_distance
        result[f"{model}_model_point"] = calculate_new_coordinates(
            latest_row.geometry.x,
            latest_row.geometry.y,
            transect_metadata[transect_ID]["Azimuth"],
            distance_difference,
        )
        result[f"{model}_model_predicted_distance"] = predicted_distance
        result[f"{model}_model_distance"] = distance_difference
        results.append(result)
    results = gpd.GeoDataFrame(results)
    return results

In [7]:
#Projection file is created here with the stats and coordinate points in table format
results = predict(gdf, linear_models, transect_metadata)
results

,TransectID,slope,intercept,group,r2_score,mae,mse,rmse,BaselineID,Year,ocean_point,linear_model_point,linear_model_predicted_distance,linear_model_distance
0,-2.147484e+09,-0.119532,-53.507306,0.0,0.029211,14.119566,331.024402,18.194076,1.0,2100,POINT (1603034.0170216693 5444035.169680288),POINT (1602810.0699988382 5444475.163140053),-89.366911,-6.293089


In [8]:
#Spatial reference added to the results
results.set_geometry("linear_model_point", inplace=True, crs=2193)
results

,TransectID,slope,intercept,group,r2_score,mae,mse,rmse,BaselineID,Year,ocean_point,linear_model_point,linear_model_predicted_distance,linear_model_distance
0,-2.147484e+09,-0.119532,-53.507306,0.0,0.029211,14.119566,331.024402,18.194076,1.0,2100,POINT (1603034.0170216693 5444035.169680288),POINT (1602810.070 5444475.163),-89.366911,-6.293089


In [9]:
#Line and polygon shapefiles are created here 
def prediction_results_to_line_polygon(results: gpd.GeoDataFrame):
    lines = []
    polygons = []
    for group_name, group_data in results.groupby(["BaselineID", "group"]):
        if len(group_data) > 1:
            # Convert the points to LineString
            line = LineString(list(group_data.geometry))
            lines.append(line)
            # Convert the points to a closed Polygon
            polygon = Polygon(list(group_data.geometry) + list(group_data.ocean_point)[::-1])
            polygons.append(polygon)
    lines = gpd.GeoSeries(lines, crs=2193)
    polygons = gpd.GeoSeries(polygons, crs=2193)
    return lines, polygons

In [10]:
lines, poly = prediction_results_to_line_polygon(results)

In [ ]:
#Saving line and polygon projection file to Z drive. Change file location accordingly  
lines, poly = prediction_results_to_line_polygon(results)
lines.to_file("Z:\DSAS\Region\Southland\BigBay\BigBay_projection_output_lines.shp")
poly.to_file("Z:\DSAS\Region\Southland\BigBay\BigBay_projection_output_polygon.shp")

In [ ]:
#Saving line and polygon projection file to folder in VS Code. Change file location accordingly
lines, poly = prediction_results_to_line_polygon(results)
lines.to_file("TEST\Southland_BigBay\BigBay_projection_output_lines.shp")
poly.to_file("TEST\Southland_BigBay\BigBay_projection_output_polygon.shp")

In [11]:
#Quick visualisation of projected polygon and historic shorelines 
m = poly.explore(tiles="Esri.WorldImagery")
gpd.GeoDataFrame(results.drop(columns=["ocean_point", "linear_model_point"]), geometry=results.linear_model_point).explore(m=m)
gdf.explore("Year", legend=True, m=m)

ValueError: Location values cannot contain NaNs.